In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import numpy.ma as ma
import matplotlib
import matplotlib.pyplot as plt
%matplotlib notebook
import matplotlib.cm as cm
import matplotlib.ticker as ticker
import matplotlib.dates as dates
from matplotlib import dates
from mpl_toolkits.axes_grid1 import ImageGrid, make_axes_locatable, host_subplot
#from mpl_toolkits.basemap import Basemap
from datetime import datetime, timedelta
import sys
import os
import pyPIPS.utils as utils
import pyPIPS.thermolib as thermo
import pyPIPS.DSDlib as dsd
#import pyPIPS.disdrometer_module as dis
import pyPIPS.plotmodule as PIPSplot
#import pyPIPS.simulator as sim
import pyPIPS.pips_io as pipsio
import pyPIPS.PIPS as pips
import pyPIPS.parsivel_params as pp
import pyPIPS.parsivel_qc as pqc
import pyPIPS.radarmodule as radar
import pyPIPS.polarimetric as dualpol
#from pyCRMtools.modules import plotmodule as plotmod
from pyCRMtools.modules import utils as CRMutils
import pandas as pd
import xarray as xr
import glob
import numpy.random as random
from scipy.stats import gamma, uniform
from scipy.special import gamma as gammafunc
from scipy import ndimage
from scipy import interpolate
from metpy.plots import StationPlot
import metpy.calc as mpcalc
from metpy.calc import wind_components
from metpy.cbook import get_test_data
from metpy.plots import StationPlot
from metpy.plots.wx_symbols import current_weather, sky_cover
from metpy.units import units
from scipy.signal import medfilt2d
import pyart
import cartopy.crs as ccrs
from IPython.display import HTML
%matplotlib inline
# %matplotlib notebook
import warnings;
warnings.filterwarnings('ignore')

In [ ]:
# Function definitions
def roundPartial(value, resolution, decimals=4):
    return np.around(np.round(value / resolution) * resolution, decimals=decimals)

def mtokm(val,pos):
    """Convert m to km for formatting axes tick labels"""
    val=val/1000.0
    return '%i' % val

In [ ]:
# Read in the radar data
radar_name = 'KGWX'
radar_type= 'NEXRAD'
elev_angle = 0.5
elev_str = str(elev_angle).replace('.', 'p')

# For 03/30/22 case (PERiLS-2022 IOP2)
date = '0330'
radar_start_datetimestamp = '20220330220424'
radar_end_datetimestamp = '20220331035633'

# Create datetime objects for start and end times
datetime_start = datetime.strptime(radar_start_datetimestamp, '%Y%m%d%H%M%S')
datetime_end = datetime.strptime(radar_end_datetimestamp, '%Y%m%d%H%M%S')

radar_basedir = \
    '/Users/dawson29/Projects/PERiLS/obsdata/2022/NEXRAD/IOP2/KGWX'
#radar_basedir = os.path.join(radar_basedir, '{}/{}'.format(date, radar_name[1:]))
radar_input_dir = os.path.join(radar_basedir, 'extracted_sweeps')
radar_output_dir = os.path.join(radar_basedir, 'extracted_sweeps')
if not os.path.exists(radar_output_dir):
    os.makedirs(radar_output_dir)

radar_start_timestamp = datetime_start.strftime('%Y%m%d%H%M')
radar_end_timestamp = datetime_end.strftime('%Y%m%d%H%M')

radar_paths = glob.glob(radar_input_dir + f'/{radar_name}*el{elev_str}_filt.nc')

radar_paths = sorted(radar_paths)
radar_input_list = []

for path in radar_paths:
    filename = os.path.basename(path)
    file_timestamp = filename[4:19]
    # print(file_timestamp)
    file_datetime = datetime.strptime(file_timestamp, '%Y%m%d_%H%M%S')
    if file_datetime >= datetime_start and file_datetime <= datetime_end:
        radar_input_list.append(path)

# Read in the individual sweeps
radarobj_list = []
for radar_path in radar_input_list:
    print(f"Reading {os.path.basename(radar_path)}")
    radarobj = pyart.io.read(radar_path)
    radarobj_list.append(radarobj)

In [ ]:
ref_varname = 'REF'
zdr_varname = 'ZDR'
rhv_varname = 'RHO'

In [ ]:
# Apply retrieval to get N0, alpha, and lamda
# Set retrieval lookup tables to use
# retrieval_tags = ['Z01', 'Z01_4dB', 'C08', 'TMM246_F_qc', 'SATP_TMM246_qc', 'TMM246_F_RB15_qc', 
#                   'SATP_TMM246_RB15_qc']
retrieval_tags = ['Z01_4dB']
lookup_basedir = '/Users/dawson29/Projects/pyPIPS/data/lookups/'
lookup_dirs = [os.path.join(lookup_basedir, tag) for tag in retrieval_tags]

for radar_path, radarobj in zip(radar_input_list, radarobj_list):
    # Retrieve gamma DSD parameters from ZH and ZDR
    print("Getting ZH and ZDR fields")
    # Get the ZH and ZDR fields from the radar object
    ZH_rad = radarobj.fields[f'{ref_varname}_filtered']['data']
    ZDR_rad = radarobj.fields[f'{zdr_varname}_filtered']['data']
    for retrieval_tag, lookup_dir in zip(retrieval_tags, lookup_dirs):
        # Read in first lookup table to get the interval between reflectivity and ZDR
        lookup_path = os.path.join(lookup_dir, 'D0.csv')
        retr_table = pd.read_csv(lookup_path, sep=',', header=0, index_col='dBZ')
        # Massage the index and column labels to get rid of extraneous zeros
        # Also convert column labels from strings to floats
        retr_table.index = retr_table.index.to_series().apply(np.around, decimals=4)
        retr_table.columns = [np.around(float(col), decimals=4) for col in retr_table.columns]

        dBZ_lookup_min = retr_table.index[0]
        dBZ_lookup_max = retr_table.index[-1]
        ZDR_lookup_min = retr_table.columns[0]
        ZDR_lookup_max = retr_table.columns[-1]
        dBZ_intv = retr_table.index[1] - retr_table.index[0]
        ZDR_intv = float(retr_table.columns[1]) - float(retr_table.columns[0])

        print(dBZ_lookup_min, dBZ_lookup_max)
        print(ZDR_lookup_min, ZDR_lookup_max)

        # Replace masked entries in ZH_rad and ZDR_rad with the minimum value of the lookup table
        ZH_mask = ZH_rad.mask
        ZDR_mask = ZDR_rad.mask
        full_mask = xr.where(ZH_mask | ZDR_mask, True, False)
        ZH_rad_new = ZH_rad.filled(dBZ_lookup_min)
        ZDR_rad_new = ZDR_rad.filled(ZDR_lookup_min)
        # Now limit values to within lookup table limits
        ZH_rad_new = np.where(ZH_rad_new > dBZ_lookup_min, ZH_rad_new, dBZ_lookup_min)
        ZH_rad_new = np.where(ZH_rad_new < dBZ_lookup_max, ZH_rad_new, dBZ_lookup_max)
        ZDR_rad_new = np.where(ZDR_rad_new > ZDR_lookup_min, ZDR_rad_new, ZDR_lookup_min)
        ZDR_rad_new = np.where(ZDR_rad_new < ZDR_lookup_max, ZDR_rad_new, ZDR_lookup_max)
        # Round ZH and ZDR fields to the nearest interval
        ZH_round = roundPartial(ZH_rad_new, dBZ_intv)
        ZDR_round = roundPartial(ZDR_rad_new, ZDR_intv)
        # Get the shape of the arrays for later, so we can reshape the flattened arrays of retrieved
        # values
        ZH_shape = ZH_round.shape
        print(ZH_shape)
        ZH_flat = ZH_round.flatten()
        ZDR_flat = ZDR_round.flatten()
        print(ZH_flat.max())
        print(ZDR_flat.max())
        for retr_varname in radar.retrieval_metadata.keys():
            print("Retrieving {} using lookup tables for {}".format(retr_varname, retrieval_tag))
            lookup_path = os.path.join(lookup_dir, '{}.csv'.format(retr_varname))
            retr_table = pd.read_csv(lookup_path, sep=',', header=0, index_col='dBZ')
            # Round the indices and columns of the DataFrame (i.e. the dBZ values) to some sane
            # number of decimal places to facilitate using it as a lookup table. The floating point
            # precision gets in the way sometimes here. For example 56.4 is dumped out as
            # 56.4<some bunch of zeros>1
            retr_table.index = retr_table.index.to_series().apply(np.around, decimals=4)
            retr_table.columns = [np.around(float(col), decimals=4) for col in
                                  retr_table.columns]
            # Gah, for some reason DataFrame.lookup sometimes barfs on perfectly good floating point
            # values in columns, so convert them back to strings here. :rolleyes:
            # EDIT 11/09/2020: Now this is happening for the rows as well. Not sure why... So change the row labels
            # to strings as well.
            retr_table.index = [str(row) for row in retr_table.index]
            retr_table.columns = [str(col) for col in retr_table.columns]
            # print(list(retr_table.index))
            # print(list(retr_table.columns))
            # Ok, now retrieve the desired retrieval variable
            # for each ZH/ZDR pair in the flattened radar sweep
        #     for ZH, ZDR in zip(ZH_flat, ZDR_flat):
        #         print(ZH, ZDR)
        #         retr_val = retr_table.lookup([ZH.astype('str')], [ZDR.astype('str')])
            # Below is from https://stackoverflow.com/questions/65882258/
            # pandas-lookup-to-be-deprecated-elegant-and-efficient-alternative?noredirect=1&lq=1
            retr_vals = retr_table.to_numpy()[retr_table.index.get_indexer(ZH_flat.astype('str')), 
                                              retr_table.columns.get_indexer(ZDR_flat.astype('str'))]
            # retr_vals = retr_table.lookup(ZH_flat.astype('str'), ZDR_flat.astype('str'))
            # retr_vals = retr_table.lookup(ZH_flat, ZDR_flat)
            # Reshape back to original shape
            retr_vals_data = retr_vals.reshape(ZH_shape)
            retr_vals_data = np.where(full_mask, np.nan, retr_vals_data)
            # retr_vals_data = np.ma.masked_array(retr_vals_data, mask=full_mask)
            new_retr_varname = f'{retr_varname}_{retrieval_tag}'
            radarobj.add_field_like(f'{ref_varname}_filtered', new_retr_varname,
                                    retr_vals_data, replace_existing=True)

    orig_filename = os.path.basename(radar_path)
    new_filename = orig_filename.replace('.nc', '_retr.nc')
    new_filepath = os.path.join(radar_output_dir, new_filename)
    print(f"Saving {new_filename}...")
    pyart.io.write_cfradial(new_filepath, radarobj)
            # retr_var_da_dict[new_retr_varname] = retr_var_da